In [1]:
#  Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [2]:
# loadmat aux function to load the matlab save files
# from: https://stackoverflow.com/review/suggested-edits/21667510

import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _has_struct(elem):
        """Determine if elem is an array and if any array item is a struct"""
        return isinstance(elem, np.ndarray) and any(isinstance(
                    e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif _has_struct(elem):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif _has_struct(sub_elem):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [3]:
# Matlab code conversion

# Read matlab file instead of excelfile (easier conversion)
data = loadmat('matlab.mat')

In [4]:
# Linearize the generator prices

# genCofA=data.generator.limit(:,:,6); -> MATLAB starts with 1, so we need the index -1
# genCofB=data.generator.limit(:,:,7);
# genCofC=data.generator.limit(:,:,8);

genCofA = data['data']['generator']['limit'][:, :, 5]
genCofB = data['data']['generator']['limit'][:, :, 6]
genCofC = data['data']['generator']['limit'][:, :, 7]

In [5]:
# Options for simulation

#options 
iterlim = 1000000000
reslim = 5000000000

In [6]:
# Definir o numero de resources usados

numGen = np.arange(1, data['data']['parameterData']['resources']['numGen'] + 1)
numLoad = np.arange(1, data['data']['parameterData']['resources']['numLoad'] + 1)
numStor = np.arange(1, data['data']['parameterData']['resources']['numStor'] + 1)
numV2G = np.arange(1, data['data']['parameterData']['resources']['numV2G'] + 1)
numCStat = np.arange(1, data['data']['parameterData']['resources']['numCStat'] + 1)
numPeriod = np.arange(1, data['data']['parameterData']['resources']['period'] + 1)
numBus = np.arange(1) # forced to 1
nOwner = np.arange(1, data['data']['parameterData']['resources']['owners'] + 1)

In [7]:
# Define as strings com os recursos para mandar para GAMS

gen = np.max(numGen)
Load = np.max(numLoad)
stor = np.max(numStor)
v2g = np.max(numV2G)
cs = np.max(numCStat)
period = np.max(numPeriod)
Bus = np.max(numBus)

In [8]:
#% Definir as matrizes com os dados para o GAMS

# General Info
pMaxImp = data['data']['parameterData']['generalInfo']['P_Max_Imp']
pMaxExp = data['data']['parameterData']['generalInfo']['P_Max_Exp']
buyPrice = data['data']['parameterData']['generalInfo']['Energy_Buy_Price']
sellPrice = data['data']['parameterData']['generalInfo']['Energy_Sell_Price']

# Geradores
genLimit = data['data']['generator']['limit'][numGen[0]-1:numGen[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
genInfo = data['data']['generator']['info']

# Cargas
loadLimit = data['data']['load']['limit'][numLoad[0]-1:numLoad[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]

# Baterias
storLimit = data['data']['storage']['limit'][numStor[0]-1:numStor[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
storInfo = data['data']['storage']['info']

# Veiculos
v2gLimit = data['data']['vehicle']['limit']
v2gInfo = data['data']['vehicle']['info']

# Charging Station
csLimit = data['data']['cstation']['limit'][numCStat[0]-1:numCStat[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
csInfo = data['data']['cstation']['info']

# Connect EV to Charging Station
EV_CS_Info = data['data']['vehicle']['timeInfo']['V2GinCS']

In [9]:
# Definir os dados da rede para o GAMS

# Linhas
branchData = data['data']['network']['branch']

# Tensão Máxima
voltageMax = data['data']['parameterData']['network']['voltageMax']

# Tensão Mínima
voltageMin = data['data']['parameterData']['network']['voltageMin']

# Angulo Máximo
angleMax = data['data']['parameterData']['network']['angleMax']

# Angulo Mínimo
angleMin = data['data']['parameterData']['network']['angleMin']

# Definir a matriz das admitancias
ybus = data['data']['parameterData']['network']['ybus']
diag = data['data']['parameterData']['network']['diag']
branchID = data['data']['parameterData']['network']['branch']

In [10]:
#% Anular o custo fixo e quadratico da DG

genLimit[:, :, 3] = 0
genLimit[:, :, 5] = 0

genLimit = np.append(genLimit, np.zeros(shape=(7, 24, 4)), axis=2)

# Not necessary anymore
#genLimit[:, :, 9] = 0
#genLimit[:, :, 11] = 0

In [11]:
#** Define the fixed sets structure to use in the parameters of the gams optimization process
#set genLimitInfo 'Indicate the number of columns of information from generators'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set loadLimitInfo 'Indicate the number of columns of information from loads'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set storLimitInfo 'Indicate the number of columns of information from storage units'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set v2gLimitInfo 'Indicate the number of columns of information from electric vehicles'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set csLimitInfo 'Indicate the number of columns of information from charging stations'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

NCOLS = 12

genLimitInfo = np.arange(1, NCOLS+1)
loadLimitInfo = np.arange(1, NCOLS+1)
storLimitInfo = np.arange(1, NCOLS+1)
v2gLimitInfo = np.arange(1, NCOLS+1)
csLimitInfo = np.arange(1, NCOLS+1)

#genLimitInfo = np.arange(1, genLimit.shape[2]+1)
#loadLimitInfo = np.arange(1, loadLimit.shape[2]+1)
#storLimitInfo = np.arange(1, storLimit.shape[2]+1)
#v2gLimitInfo = np.arange(1, v2gLimit.shape[2]+1)
#csLimitInfo = np.arange(1, csLimit.shape[0]+1)

In [12]:
# * Define the several sets structure used in the gams optimization process, 
# these sets depends the number of resources used in matlab code

_t = np.arange(1, period+1)
_gen = np.arange(1, gen+1)
_Load = np.arange(1, Load+1)
_stor = np.arange(1, stor+1)
_v2g = np.arange(1, v2g+1)
_cs = np.arange(1, cs+1)

In [13]:
# Aux function to convert numpy into dictionaries

def _auxDictionary(a):
    temp_dictionary = {}
    
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
                
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
                
    return temp_dictionary

#temp_dict1 = _auxDictionary(loadLimit)

# Plot results

fig, axs = plt.subplots(nrows=7, ncols=7, figsize=(24, 20))

for i in np.arange(7):
    # Generators
    axs[i, 0].plot(result_genActPower.values[i])
    axs[i, 0].plot(result_genExcActPower.values[i])
    axs[i, 0].legend(['genActPower', 'genExcActPower'])
    
for i in np.arange(6):
    # Loads
    axs[i, 1].plot(loadLimit[:, :, 0][i, :])
    axs[i, 1].plot(result_loadRedActPower.values[i])
    axs[i, 1].plot(result_loadCutActPower.values[i])
    axs[i, 1].plot(result_loadENS.values[i])
    axs[i, 1].legend(['loadActPower', 'loadRedActPower', 'loadCutActPower', 'loadENS'])


axs[0, 2].plot(result_pimp.values)
axs[0, 2].plot(result_pexp.values)
axs[0, 2].legend(['pImp', 'pExp'])


for i in range(3):
    axs[i, 3].plot(result_storEnerState.values[i])
    axs[i, 3].legend(['storEnerState'])
    
for i in range(3):
    axs[i, 4].plot(result_storDchActPower.values[i])
    axs[i, 4].legend(['storDchActPower'])
    
for i in range(3):
    axs[i, 5].plot(result_storChActPower.values[i])
    axs[i, 5].legend(['storChActPower'])

for i in range(5):
    axs[i, 6].plot(result_v2gEnerState.values[i])
    axs[i, 6].legend(['v2gEnerState'])


plt.show()

In [14]:
# Pymoo import

import pymoo
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.termination.default import DefaultSingleObjectiveTermination

from pymoo.core.repair import Repair

In [15]:
# Class to speed up code execution

class HeuristicData(object):
    def __init__(self):
        return
    
    
    def objectiveFunction(self):
        return
    
    
    def doIteration(self):
        return
    
    
    def newIteration(self):
        return
    

class EnergyCommunity(HeuristicData):
    
    def __init__(self, parameters, variables):
        # Set initial values
        self.__initial_variables__ = variables
        self.__var_idx__ = [variables[v].ravel().shape[0] for v in variables.keys()]
        self.__var_names__ = list(variables.keys())
        
        # Value assignments
        self.parameterAssign(parameters)
        self.variableAssign(variables)
        
        self.encoded = None
        self.decoded = None
        
        
        self.objFn = 0
        
        self.checkV2G()
        self.checkCS()
        return
    
    
    def parameterAssign(self, parameters):
        self.genLimit = parameters['genLimit']
        self.genInfo = parameters['genInfo']
        self.pImpMax = parameters['pImpMax']
        self.pExpMax = parameters['pExpMax']
        self.loadLimit = parameters['loadLimit']
        self.loadActPower = parameters['loadActPower']
        self.storLimit = parameters['storLimit']
        self.storInfo = parameters['storInfo']
        self.v2gLimit = parameters['v2gLimit']
        self.v2gInfo = parameters['v2gInfo']
        self.csLimit = parameters['csLimit']
        self.csInfo = parameters['csInfo']
        self.EV_CS_Info = parameters['EV_CS_Info']
        self.buyPrice = parameters['buyPrice']
        self.sellPrice = parameters['sellPrice']
        self.t = parameters['t']
        self.gen = parameters['gen']
        self.load = parameters['load']
        self.stor = parameters['stor']
        self.v2g = parameters['v2g']
        self.cs = parameters['cs']
        #self.storPieceSegments = parameters['storPieceSegments']
        self.storCapCost = parameters['storCapCost']
        #self.storCyclesMult = parameters['storCyclesMult']
        self.v2gCapCost = parameters['v2gCapCost']
        #self.v2gCyclesMult = parameters['v2gCyclesMult']
        
        return
    
    
    def variableAssign(self, variables):
        self.genActPower = variables['genActPower']
        self.genExcActPower = variables['genExcActPower']
        self.pImp = variables['pImp']
        self.pExp = variables['pExp']
        self.loadRedActPower = variables['loadRedActPower']
        self.loadENS = variables['loadENS']
        self.storDchActPower = variables['storDchActPower']
        self.storChActPower = variables['storChActPower']
        self.EminRelaxStor = variables['EminRelaxStor']
        #self.v2gDchActPower = variables['v2gDchActPower']
        #self.v2gChActPower = variables['v2gChActPower']
        self.genXo = variables['genXo']
        self.loadXo = variables['loadXo']
        self.storDchXo = variables['storDchXo']
        self.storChXo = variables['storChXo']
        self.v2gDchXo = variables['v2gDchXo']
        self.v2gChXo = variables['v2gChXo']
        #self.storSegSoC = self.variables['storSegSoC']
        #self.v2gSegSoC = self.variables['v2gSegSoC']
        #self.storSegXo = self.variables['storSegXo']
        #self.v2gSegXo = self.variables['v2gSegXo']
        
        self.loadCutActPower = np.zeros((len(self.load),
                                         len(self.t)))
        
        self.storEnerState = np.zeros((len(self.stor),
                                       len(self.t)))
        
        #self.v2gEnerState = np.zeros((len(self.v2g),
        #                              len(self.t)))
        
        #self.EminRelaxEv = np.zeros((len(self.v2g),
        #                             len(self.t)))
        
        return
    
    
    def objectiveFunction(self):
        i: cython.int
        t: cython.int
        g: cython.int
        l: cython.int
        s: cython.int
        v: cython.int
            
        t_range: cython.int[self.t.shape[0]] = range(self.t.shape[0])
        gen_range: cython.int[self.gen.shape[0]] = range(self.gen.shape[0])
        load_range: cython.int[self.load.shape[0]] = range(self.load.shape[0])
        stor_range: cython.int[self.stor.shape[0]] = range(self.stor.shape[0])
        v2g_range: cython.int[self.v2g.shape[0]] = range(self.v2g.shape[0])
            
        balance_gens: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_loads: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_stor: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_v2g: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
            
        for t in t_range:
            balance_gens[t] = np.sum([self.genActPower[g, t] - self.genExcActPower[g, t]
                                      for g in gen_range])
            
            balance_loads[t] = np.sum([self.loadActPower[l, t] - self.loadRedActPower[l, t] - self.loadCutActPower[l, t] - self.loadENS[l, t] 
                                       for l in load_range])
            
            balance_stor[t] = np.sum([self.storChActPower[s, t] - self.storDchActPower[s, t]
                                      for s in stor_range])
            
            balance_v2g[t] = np.sum([self.v2gChActPower[v, t] - self.v2gDchActPower[v, t]
                                     for v in v2g_range])
            
        balance_rest: cython.float[self.t.shape[0]] = balance_gens - balance_loads - balance_stor - balance_v2g
            
        # Balance and import/export penalties
        mask = balance_rest > 0
        self.pImp[mask] *= 0
        self.pExp[mask] = balance_rest[mask]
        
        mask = balance_rest < 0
        self.pExp[mask] *= 0
        self.pImp[mask] = abs(balance_rest)[mask]
        
        
        # Attribute penalties for import/export
        balance_penalty: cython.float = 0
        for t in t_range:
            if (self.pImp[t] > self.pImpMax[t]):
                balance_penalty += 100000
                
            if (self.pExp[t] > self.pExpMax[t]):
                balance_penalty += 100000
                
            
        temp_gens: cython.float = sum([self.genActPower[g, t] * self.genLimit[g, t, 2] + self.genExcActPower[g, t] * self.genLimit[g, t, 4]
                                       for t in t_range for g in gen_range])
            
        temp_loads: cython.float = sum([self.loadRedActPower[l, t] * self.loadLimit[l, t, 6] + self.loadCutActPower[l, t] * self.loadLimit[l, t, 7] + self.loadENS[l, t] * self.loadLimit[l, t, 9] 
                                        for t in t_range for l in load_range])
            
        #temp_stor: cython.float = sum([self.storDchActPower[s, t] * self.storLimit[s, t, 3] - self.storChActPower[s, t] * self.storLimit[s, t, 2] + self.EminRelaxStor[s, t] * 200 
        #                               for t in t_range for s in stor_range])
            
        #temp_v2g: cython.float = sum([self.v2gDchActPower[v, t] * self.v2gLimit[v, t, 6] - self.v2gChActPower[v, t] * self.v2gLimit[v, t, 5] + self.EminRelaxEv[v, t] * 200
        #                              for t in t_range for v in v2g_range])
            
        temp_rest: cython.float = sum([self.pImp[t] * self.buyPrice[t] - self.pExp[t] * self.sellPrice[t] 
                                       for t in t_range])   
        
        temp_storDeg = sum([self.storCapCost[s] * (self.storEnerState[s, t]/self.storInfo[s, 5] - 0.63)**2 + self.storDchActPower[s, t] * self.storLimit[s, t, 3] + self.storChActPower[s, t] * self.storLimit[s, t, 2] + (6.5e-3)/self.storInfo[s, 5] * self.storChActPower[s, t]**2
                            for t in t_range for s in stor_range])
        
        temp_v2gDeg = sum([self.v2gCapCost[v] * (self.v2gEnerState[v, t]/self.v2gInfo[v, 4] - 0.63)**2 + self.v2gDchActPower[v, t] * self.v2gLimit[v, t, 6] + self.v2gChActPower[v, t] * self.v2gLimit[v, t, 5] + (6.5e-3)/self.v2gInfo[v, 4] * self.v2gChActPower[v, t]**2
                           for t in t_range for v in v2g_range])
            
        #temp_storDeg: cython.float = sum([self._auxStorDegCost(s, t) for t in t_range for s in stor_range])
            
        #temp_v2gDeg: cython.float = sum([self._auxV2GDegCost(v, t) for t in t_range for v in v2g_range])
        
        #self.objFn = (temp_gens + temp_loads + temp_stor + temp_v2g + temp_rest)*1.0 + balance_penalty + temp_storDeg# + temp_v2gDeg
        self.objFn = temp_gens + temp_loads + temp_rest + balance_penalty + temp_storDeg + temp_v2gDeg

        return
    
    
    def doIteration(self):
        self.checkImpExp()
        self.checkGens()
        self.checkLoads()
        self.checkStor()
        #self.new_checkV2G()
        #self.checkCS()
        
        self.objectiveFunction()
        
        self.encoded = self.encode()
        return
    
    
    def newIteration(self, encoded_vals):
        self.variableAssign(self.decode(encoded_vals))
        self.doIteration()
        return

    
    def checkImpExp(self):
        # Imports
        mask = self.pImp >= self.pImpMax
        self.pImp[mask] = self.pImpMax[mask]
        
        mask = self.pImp < 0
        self.pImp[mask] *= 0
        
        # Exports
        mask = self.pExp > self.pExpMax
        self.pExp[mask] = self.pExpMax[mask]
        
        mask = self.pExp < 0
        self.pExp[mask] *= 0
        
        return
    
    
    def checkGens(self):
        self.genXo = (self.genXo > 0.5).astype(int)
        
        # Maximum generation
        mask = self.genActPower > self.genLimit[:, :, 0]
        self.genActPower[mask] = self.genLimit[:, :, 0][mask]
        
        mask = self.genActPower < 0
        self.genActPower[mask] *= 0
    
        
        # Generator types - type 1
        mask = self.genInfo[:, 1] == np.ones((self.genInfo[:, 4].shape))
        self.genActPower[mask] = (self.genLimit[:, :, 0] * self.genXo)[mask]
        
        # Generator types - type 2
        mask = self.genInfo[:, 1] == 2*np.ones((self.genInfo[:, 4].shape))
        self.genActPower[mask] = self.genLimit[:, :, 0][mask]
    
        return
    
    
    def checkLoads(self):
        
        # Bound the values to either 0 or 1
        self.loadXo = (self.loadXo > 0.5).astype(int)
        
        # Build a matrix of ones
        temp_vals = np.ones((self.loadRedActPower.shape[0],
                             self.loadRedActPower.shape[1]))
        
        # RedAct value checks
        mask = self.loadRedActPower > temp_vals * self.loadLimit[:, :, 2]
        self.loadRedActPower[mask] = (temp_vals * self.loadLimit[:, :, 2])[mask]
        
        mask = self.loadRedActPower < 0
        self.loadRedActPower[mask] *= 0
        
        # loadCut value checks
        self.loadCutActPower = self.loadLimit[:, :, 3] * self.loadXo
        #self.loadCutActPower *= self.loadXo
        
        # loadENS
        temp_vals = self.loadActPower - self.loadRedActPower - self.loadCutActPower
        mask = self.loadENS > temp_vals
        self.loadENS[mask] = temp_vals[mask]
        return
    
    
    def checkStor(self):
        
        # Binary variables bound
        self.storChXo = (self.storChXo > 0.5).astype(int)
        self.storDchXo = (self.storDchXo > 0.5).astype(int)
        
        mask = None
        # Discharge inequality
        mask = self.storDchActPower > self.storLimit[:, :, 1]
        self.storDchActPower[mask] = self.storLimit[:, :, 1][mask]
        
        mask = self.storDchActPower <= 0
        self.storDchActPower[mask] *= 0
        
        # Charge inequality
        mask = self.storChActPower > self.storLimit[:, :, 0]
        self.storChActPower[mask] = self.storLimit[:, :, 0][mask]
        
        mask = self.storChActPower <= 0
        self.storChActPower[mask] *= 0
        
        # Initial stor SoC
        self.storEnerState[:, 0] = self.storInfo[:, 5] * (self.storInfo[:, 9]/100) + self.storChActPower[:, 0] * (self.storInfo[:, 7]/100) - self.storDchActPower[:, 0]/(self.storInfo[:, 8]/100)
        
        t: cython.int
        t_range: cython.int[len(self.t) - 1] = range(1, len(self.t))
        # Fix the timestep dependencies
        for t in t_range:
            # Check if charging
            mask = self.storChXo[:, t] > np.zeros(self.storChXo[:, t].shape)
            charged = self.storChActPower[:, t] * (self.storInfo[:, 7]/100)
            # Prevent charging beyong limit
            secondary_mask = (self.storEnerState[:, t-1] + charged) > self.storInfo[:, 5]
            self.storChActPower[:, t][secondary_mask] = ((self.storInfo[:, 5] - self.storEnerState[:, t-1]) / (self.storInfo[:, 7]/100))[secondary_mask]
            
            # Check if discharging
            mask = self.storDchXo[:, t] > np.zeros(self.storDchXo[:, t].shape)
            discharged = self.storDchActPower[:, t] / (self.storInfo[:, 8]/100)
            secondary_mask = (self.storEnerState[:, t-1] - discharged) < 0
            self.storDchActPower[:, t][secondary_mask] = (self.storEnerState[:, t-1] * (self.storInfo[:, 8]/100))[secondary_mask]
            
            self.storChActPower[:, t] *= self.storChXo[:, t]
            self.storDchActPower[:, t] *= self.storDchXo[:, t]
            self.storEnerState[:, t] = self.storEnerState[:, t-1] + (self.storChActPower[:, t] * (self.storInfo[:, 7]/100)) - (self.storDchActPower[:, t]/(self.storInfo[:, 8]/100))
            
            mask = self.storEnerState[:, t] < self.storInfo[:, 6] * (self.storInfo[:, 5] / 100) - self.EminRelaxStor[:, t]
            self.storEnerState[:, t][mask] = (self.storInfo[:, 6] * (self.storInfo[:, 5] / 100) - self.EminRelaxStor[:, t])[mask]
        return
    
    
    def checkV2G_old(self):
        
        # Binary variables bounding
        self.v2gChXo = (self.v2gDchXo < 0.33).astype(int)
        self.v2gDchXo = (self.v2gDchXo > 0.66).astype(int)
        
        mask = None
        t: cython.int
        t_range: cython.int[len(self.t) - 1] = range(1, len(self.t))
        for t in t_range:
            mask = self.v2gEnerState[:, t-1] == self.v2gInfo[:, 4]
            self.v2gChXo[:, t][mask] *= 0
            self.v2gChActPower[:, t][mask] *= 0
            
            mask = self.v2gEnerState[:, t-1] == 0
            self.v2gDchXo[:, t][mask] *= 0
            self.v2gDchActPower[:, t][mask] *= 0
            
        # Discharge rate
        temp_vals = self.v2gLimit[:, :, 4] * self.v2gLimit[:, :, 0] * self.v2gDchXo
        mask = self.v2gDchActPower > temp_vals
        self.v2gDchActPower[mask] = temp_vals[mask]
        
        
        
        # Charge rate
        temp_vals = self.v2gLimit[:, :, 3] * self.v2gLimit[:, :, 0] * self.v2gChXo
        mask = self.v2gChActPower > temp_vals
        self.v2gChActPower[mask] = temp_vals[mask]
        
        # Energy Balance Initial Eq
        mask = self.v2gLimit[:, 0, 0] == 0
        temp_vals = self.v2gLimit[:, 0, 1] + (self.v2gChActPower[:, 0] * self.v2gInfo[:, 7]) - (self.v2gDchActPower[:, 0] / self.v2gInfo[:, 8])
        self.v2gEnerState[:, 0][mask] = temp_vals[mask]
        
        # Energy Balance 1
        mask = self.v2gLimit[:, :-1, 0] == 1
        secondary_mask = self.v2gLimit[:, 1:, 0] == 1
        
        temp_vals = [self.v2gEnerState[:, t-1] + self.v2gLimit[:, t, 1] + (self.v2gChActPower[:, t] * self.v2gInfo[:, 7]) - (self.v2gDchActPower[:, t] / self.v2gInfo[:, 8])
                     for t in range(1, len(self.t))]
        temp_vals = np.array(temp_vals).transpose()
        self.v2gEnerState[:, 1:][mask & secondary_mask] = temp_vals[mask & secondary_mask]
        
        # Energy Balance 2
        mask = self.v2gLimit[:, :-1, 0] == 0
        temp_vals = [self.v2gLimit[:, t, 1] + (self.v2gChActPower[:, t] * self.v2gInfo[:, 8]) + (self.v2gDchActPower[:, t] / self.v2gInfo[:, 8])
                     for t in range(1, len(self.t))]
        temp_vals = np.array(temp_vals).transpose()
        self.v2gEnerState[:, 1:][mask & secondary_mask] = temp_vals[mask & secondary_mask]
        return
    
    
    def new_checkV2G(self):
        self.v2gDchXo = (self.v2gDchXo > 0.5).astype(int)
        self.v2gChXo = (self.v2gChXo > 0.5).astype(int)
        
        mask = self.v2gDchXo + self.v2gChXo == 2*np.ones(self.v2gDchXo.shape)
        self.v2gDchXo[mask] *= 0
        
        # Discharge rate
        mask = self.v2gDchActPower > self.v2gLimit[:, :, 4] * self.v2gLimit[:, :, 0] * self.v2gDchXo
        self.v2gDchActPower[mask] = (self.v2gLimit[:, :, 4] * self.v2gLimit[:, :, 0] * self.v2gDchXo)[mask]
        
        # Charge rate
        mask = self.v2gChActPower > self.v2gLimit[:, :, 3] * self.v2gLimit[:, :, 0] * self.v2gChXo
        self.v2gChActPower[mask] = (self.v2gLimit[:, :, 3] * self.v2gLimit[:, :, 0] * self.v2gChXo)[mask]
        
        # Energy Limits
        mask = self.v2gEnerState > (np.ones(self.v2gEnerState.shape).transpose() * self.v2gInfo[:, 4]).transpose()
        self.v2gEnerState[mask] = (np.ones(self.v2gEnerState.shape).transpose() * self.v2gInfo[:, 4]).transpose()[mask]
        
        # bat balance zero
        mask = self.v2gLimit[:, :, 0] == np.zeros(self.v2gLimit[:, :, 0].shape)
        self.v2gEnerState[mask] *= 0
        
        # bat balance initial eq
        mask = self.v2gLimit[:, 0, 0] == np.ones(self.v2gLimit[:, 0, 0].shape)
        self.v2gLimit[:, 0, 0][mask] = (self.v2gLimit[:, 0, 2] + self.v2gChActPower[:, 0] * self.v2gInfo[:, 7] - (self.v2gDchActPower[:, 0] / self.v2gInfo[:, 8]))[mask]
        
        # batery update
        t: cython.int
        t_range: cython.int[len(self.t) - 1] = range(1, len(self.t))
        for t in t_range:
            mask = (self.v2gLimit[:, t-1, 0] + self.v2gLimit[:, t, 0]) == 2*np.ones(self.v2gLimit[:, t, 0].shape)
            self.v2gEnerState[:, t][mask] = (self.v2gEnerState[:, t-1] + self.v2gLimit[:, t, 1] + self.v2gChActPower[:, t] * self.v2gInfo[:, 7] - (self.v2gChActPower[:, t] / self.v2gInfo[:, 8]))[mask]
            
            mask = self.v2gLimit[:, t-1, 0] == np.zeros(self.v2gLimit[:, t-1, 0].shape)
            secondary_mask = self.v2gLimit[:, t, 0] == np.ones(self.v2gLimit[:, t, 0].shape)
            self.v2gEnerState[:, t][mask & secondary_mask] = (self.v2gLimit[:, t, 1] + self.v2gChActPower[:, t] * self.v2gInfo[:, 7] + (self.v2gDchActPower[:, t] / self.v2gInfo[:, 8]))[mask & secondary_mask]
        
        
        # Energy Limits -> again after changing the SOC
        mask = self.v2gEnerState > (np.ones(self.v2gEnerState.shape).transpose() * self.v2gInfo[:, 4]).transpose()
        self.v2gEnerState[mask] = (np.ones(self.v2gEnerState.shape).transpose() * self.v2gInfo[:, 4]).transpose()[mask]
        return
    
    
    def checkV2G(self):
        # Binary variables bounding
        #self.v2gChXo = (self.v2gDchXo < 0.33).astype(int)
        #self.v2gDchXo = (self.v2gDchXo > 0.66).astype(int)
        
        self.v2gDchActPower = np.zeros((len(self.v2g),
                                        len(self.t)))
        
        self.v2gChActPower = np.zeros((len(self.v2g),
                                       len(self.t)))
        
        self.v2gEnerState = np.zeros((len(self.v2g),
                                      len(self.t)))
        
        self.EminRelaxEv = np.zeros((len(self.v2g),
                                     len(self.t)))
        
        # Bound binary variables
        self.v2gChXo = (self.v2gChXo > 0.5).astype(int)
        self.v2gDchXo = (self.v2gDchXo > 0.5).astype(int)
        #self.v2gChXo = 1 - self.v2gDchXo

        mask = None

        v: cython.int
        c: cython.int

        # Discharge rate
        #temp_val = par['v2gLimit'][:, :, 4] * par['v2gLimit'][:, :, 0]
        #mask = pop['v2gDchActPower'] > temp_val
        #pop['v2gDchActPower'][mask] = temp_val[mask]


        # Charge rate
        #temp_val = par['v2gLimit'][:, :, 3] * par['v2gLimit'][:, :, 0]
        #mask = pop['v2gChActPower'] > temp_val
        #pop['v2gChActPower'][mask] = temp_val[mask]

        self.v2gDchActPower = self.v2gLimit[:, :, 4] * self.v2gLimit[:, :, 0]
        self.v2gChActPower = self.v2gLimit[:, :, 3] * self.v2gLimit[:, :, 0]

        # V2G constraints
        v_range: cython.int[len(self.v2g)] = range(len(self.v2g))
        c_range: cython.int[len(self.cs)] = range(len(self.cs))
        for v in v_range:
            # Check connection to charging stations
            isConnected = False
            connectedTo = 0

            # Check the charging stations
            for c in c_range:
                isConnected = True
                if self.EV_CS_Info[v, c, 0] > 0:
                    self.v2gChActPower[v, 0] = min(self.v2gChActPower[v, 0], self.csInfo[c, 4])
                    self.v2gDchActPower[v, 0] = min(self.v2gDchActPower[v, 0], self.csInfo[c, 5])

                    connectedTo = c
                else:
                    self.v2gDchXo[v, 0] = 0
                    self.v2gChXo[v, 0] = 0

            if self.v2gChXo[v, 0] + self.v2gDchXo[v, 0] > 1:
                self.v2gDchXo[v, 0] = 1 - self.v2gChXo[v, 0]

            mask = self.v2gLimit[v, :, 2] > 0
            temp_val = self.v2gLimit[v, :, 2][mask]

            if isConnected & (len(temp_val) > 0):
                if self.v2gEnerState[v, 0] < temp_val[0]:
                    next_index = next((idx for idx, val in np.ndenumerate(self.v2gLimit[v, :, 2]) 
                                       if val==temp_val[0]))[0]
                    min_tsteps = np.ceil((temp_val[0] - self.v2gEnerState[v, 0]) / self.csInfo[connectedTo, 4]) - 1

                    if min_tsteps >= next_index:
                        self.v2gChXo[v, 0] = 1
                        self.v2gDchXo[v, 0] = 0

            self.v2gChActPower[v, 0] *= self.v2gChXo[v,  0]
            self.v2gDchActPower[v, 0] *= self.v2gDchXo[v, 0]

            if self.v2gLimit[v, 0, 0] == 0:
                self.v2gEnerState[v, 0] = 0
            elif self.v2gLimit[v, 0, 0] == 1:
                self.v2gEnerState[v, 0] = self.v2gLimit[v, 0, 1] + self.v2gChActPower[v, 0] * self.v2gInfo[v, 7] - self.v2gDchActPower[v, 0] / self.v2gInfo[v, 8]

            #print('v2gEnerState: {}'.format(pop['v2gEnerState'][v, 0]))

            # Timestep
            for t in range(1, len(self.t)):

                isConnected = False
                connectedTo = 0

                # Check the charging stations
                for c in c_range:
                    if self.EV_CS_Info[v, c, t] > 0:
                        isConnected = True
                        self.v2gChActPower[v, t] = min(self.v2gChActPower[v, t], self.csInfo[c, 4])
                        self.v2gDchActPower[v, t] = min(self.v2gDchActPower[v, t], self.csInfo[c, 5])

                        connectedTo = c
                    else:
                        self.v2gDchXo[v, t] = 0
                        self.v2gChXo[v, t] = 0


                # Disable charge and discharge in the same period
                if self.v2gChXo[v, t] + self.v2gDchXo[v, t] > 1:
                    self.v2gDchXo[v, t] = 1 - self.v2gChXo[v, t]

                # Incentivise charge to meet minimum limits
                mask = self.v2gLimit[v, t:, 2] > 0
                temp_val = self.v2gLimit[v, t:, 2][mask]

                # Check if there are any requirements for EVs
                if isConnected & (len(temp_val) > 0):
                    if self.v2gEnerState[v, t-1] < temp_val[0]:
                        next_index = next((idx for idx, val in np.ndenumerate(self.v2gLimit[v, t:, 2]) 
                                           if val==temp_val[0]))[0]
                        min_tsteps = np.ceil((temp_val[0] - self.v2gEnerState[v, t-1])  /  self.csInfo[connectedTo, 4]) - 1
                        if min_tsteps <= next_index:
                            self.v2gChXo[v, t] = 1
                            self.v2gDchXo[v, t] = 0



                            if (self.v2gEnerState[v, t-1] + (self.v2gChActPower[v, t] * float(self.v2gInfo[v, 7]))) >= self.v2gInfo[v, 4]:
                                self.v2gChActPower[v, t] = (self.v2gInfo[v, 4] - self.v2gEnerState[v, t-1]) / float(self.v2gInfo[v, 7])

                # Prevent charging when battery is full
                if self.v2gEnerState[v, t-1] == self.v2gInfo[v, 4]:
                    #print('HERE')
                    self.v2gChXo[v, t] = 0

                # Prevent discharge when battery is empty
                elif self.v2gEnerState[v, t-1] == 0:
                    self.v2gDchXo[v, t] = 0

                self.v2gChActPower[v, t] *= self.v2gChXo[v, t]
                self.v2gDchActPower[v, t] *= self.v2gDchXo[v, t]

                # Update battery capacity
                if (self.v2gLimit[v, t-1, 0] == 1) & (self.v2gLimit[v, t, 0] == 1):
                    self.v2gEnerState[v, t] = self.v2gEnerState[v, t-1] + self.v2gLimit[v, t, 1] + (self.v2gChActPower[v, t] * float(self.v2gInfo[v, 7])) - (self.v2gDchActPower[v, t] / float(self.v2gInfo[v, 8]))
                elif (self.v2gLimit[v, t-1, 0] == 0) & (self.v2gLimit[v, t, 0] == 1):
                    self.v2gEnerState[v, t] = self.v2gLimit[v, t, 1] + (self.v2gChActPower[v, t] * float(self.v2gInfo[v, 7])) + (self.v2gDchActPower[v, t] / float(self.v2gInfo[v, 8]))

        return
    
    
    def checkCS(self):
        
        self.csActPower = np.zeros((len(self.cs),
                                    len(self.t)))
        
        self.csActPowerNet = np.zeros((len(self.cs),
                                       len(self.t)))
        
        #self.v2gChXo = (self.v2gDchXo < 0.33).astype(int)
        #self.v2gDchXo = (self.v2gDchXo > 0.66).astype(int)
        
        
        c: cython.int
        t: cython.int
        v: cython.int
            
        t_range: cython.int[len(self.t)] = range(len(self.t))
        c_range: cython.int[len(self.cs)] = range(len(self.cs))
        v_range: cython.int[len(self.v2g)] = range(len(self.v2g))
        
        # Timesteps
        for t in t_range:

            # Charging station constraints
            for c in c_range:

                temp_val = 0
                temp_val2 = 0
                for v in v_range:
                    if self.EV_CS_Info[v, c, t] > 0:
                        temp_val += (self.v2gChActPower[v, t]  - self.v2gDchActPower[v, t])
                        temp_val2 += (self.v2gChActPower[v, t] / (self.csInfo[c, 6] / 100) - (self.v2gDchActPower[v, t] * self.csInfo[c, 7] / 100))


                if temp_val > self.csInfo[c, 4]:
                    temp_val = self.csInfo[c, 4]
                if temp_val < -self.csInfo[c, 5]:
                    temp_val = -self.csInfo[c, 5]

                self.csActPower[c, t] = temp_val
                self.csActPowerNet[c, t] = temp_val2
        return
    
    
    def encode(self):
        var_dictionary = {'genActPower': self.genActPower,
                          'genExcActPower': self.genExcActPower,
                          'pImp': self.pImp,
                          'pExp': self.pExp,
                          'loadRedActPower': self.loadRedActPower,
                          'loadENS': self.loadENS,
                          'storDchActPower': self.storDchActPower,
                          'storChActPower': self.storChActPower,
                          'EminRelaxStor': self.EminRelaxStor,
                          #'v2gDchActPower': self.v2gDchActPower,
                          #'v2gChActPower': self.v2gChActPower,
                          'genXo': self.genXo,
                          'loadXo': self.loadXo,
                          'storDchXo': self.storDchXo,
                          'storChXo': self.storChXo,
                          'v2gDchXo': self.v2gDchXo,
                          'v2gChXo': self.v2gChXo
                         }
        
        result_concat = np.concatenate([var_dictionary[v].ravel() for v in var_dictionary.keys()])
        return result_concat
    
    
    def decode(self, new_variables):
        result_decoded = {}
        current_index = 0
        
        for idx in range(len(self.__var_names__)):
            result_index = current_index + self.__var_idx__[idx]
            result_decoded[self.__var_names__[idx]] = np.reshape(new_variables[current_index:result_index],
                                                                 self.__initial_variables__[self.__var_names__[idx]].shape)
            
            current_index = result_index
        
        
        return result_decoded
    

In [16]:
# Turn and recover vector for encoding

def encode(var_dictionary):
    result_concat = np.concatenate([var_dictionary[v].ravel() for v in var_dictionary.keys()])
    return result_concat


def decode(var_vector, var_dictionary):
    result_decoded = {}
    var_idx = [var_dictionary[v].ravel().shape[0] for v in var_dictionary.keys()]

    current_index = 0
    dictionary_names = list(var_dictionary.keys())
    
    for idx in np.arange(len(dictionary_names)):
        result_index = current_index + var_idx[idx]
        result_decoded[dictionary_names[idx]] = np.reshape(var_vector[current_index:result_index], 
                                                           var_dictionary[dictionary_names[idx]].shape)
        
        current_index = result_index
        
    return result_decoded

In [17]:
# Positive variables

genActPower = np.ones(shape=(len(_gen), len(_t)))
genExcActPower = np.ones(shape=(len(_gen), len(_t)))
pImp = np.ones(shape=(len(_t)))
pExp = np.ones(shape=(len(_t)))

# Load variables
loadRedActPower = np.ones(shape=(len(_Load), len(_t)))
loadCutActPower = np.ones(shape=(len(_Load), len(_t)))
loadENS = np.ones(shape=(len(_Load), len(_t)))

# Storage variables
storDchActPower = np.ones(shape=(len(_stor), len(_t)))
storChActPower = np.ones(shape=(len(_stor), len(_t)))
storEminRelaxStor = np.ones(shape=(len(_stor), len(_t)))
storEnerState = np.ones(shape=(len(_stor), len(_t)))

# V2G variables
v2gDchActPower = np.ones(shape=(len(_v2g), len(_t)))
v2gChActPower = np.ones(shape=(len(_v2g), len(_t)))
v2gEnerState = np.ones(shape=(len(_v2g), len(_t)))
v2gEminRelaxEv = np.ones(shape=(len(_v2g), len(_t)))

# Charging Stations
csActPower = np.ones(shape=(len(_cs), len(_t)))
csActPowerNet = np.ones(shape=(len(_cs), len(_t)))

# Storage deg
storSegSoC = np.ones(shape=(len(_stor), len(_t), 3))
v2gSegSoC = np.ones(shape=(len(_v2g), len(_t), 3))

In [18]:
# Binary variables

genXo = np.zeros(shape=(len(_gen), len(_t)))
loadXo = np.zeros(shape=(len(_Load), len(_t)))

storChXo = np.zeros(shape=(len(_stor), len(_t)))
storDchXo = np.zeros(shape=(len(_stor), len(_t)))

v2gChXo = np.zeros(shape=(len(_v2g), len(_t)))
v2gDchXo = np.zeros(shape=(len(_v2g), len(_t)))

# Battery deg binary variables
storSegXo = np.zeros(shape=(len(_stor), len(_t), 3))
v2gSegXo = np.zeros(shape=(len(_v2g), len(_t), 3))

In [19]:
# Define dictionaries

dict_var = {'genActPower': genActPower,
            'genExcActPower': genExcActPower,
            'pImp': pImp,
            'pExp': pExp,
            'loadRedActPower': loadRedActPower,
            'loadCutActPower': loadCutActPower,
            'loadENS': loadENS,
            'storDchActPower': storDchActPower,
            'storChActPower': storChActPower,
            'EminRelaxStor': storEminRelaxStor,
            'storEnerState': storEnerState,
            'v2gDchActPower': v2gDchActPower,
            'v2gChActPower': v2gChActPower,
            'EminRelaxEV': v2gEminRelaxEv,
            'v2gEnerState': v2gEnerState,
            'csActPower': csActPower,
            'csActPowerNet': csActPowerNet,
            'genXo': genXo,
            'loadXo': loadXo,
            'storDchXo': storDchXo,
            'storChXo': storChXo,
            'v2gDchXo': v2gDchXo,
            'v2gChXo': v2gChXo}#,
            #'storSegSoC': storSegSoC,
            #'v2gSegSoC': v2gSegSoC,
            #'storSegXo': storSegXo,
            #'v2gSegXo': v2gSegXo}
encoded_vars = encode(dict_var)
encoded_vars = encoded_vars.astype(float)
decoded_vars = decode(encoded_vars, dict_var)

dict_par = {'genLimit': genLimit,
            'genInfo': genInfo,
            'pImpMax': pMaxImp,
            'pExpMax': pMaxExp,
            'loadLimit': loadLimit,
            'loadActPower': loadLimit[:, :, 0],
            'storLimit': storLimit,
            'storInfo': storInfo[:, :12],
            'v2gLimit': v2gLimit,
            'v2gInfo': v2gInfo,
            'csLimit': csLimit,
            'csInfo': csInfo,
            'EV_CS_Info': EV_CS_Info,
            'buyPrice': buyPrice,
            'sellPrice': sellPrice,
            't': _t,
            'gen': _gen,
            'load': _Load,
            'stor': _stor,
            'v2g': _v2g,
            'cs': _cs,
            #'storPieceSegments': np.array([1, 2, 3]),
            'storCapCost': storInfo[:, 5] * 0.00105,
            #'storCyclesMult': np.array([1000, 1200, 700]),
            'v2gCapCost': v2gInfo[:, 4] * 0.00105}
            #'v2gCyclesMult': np.array([1000, 1000, 1000, 1000, 1000])}
encoded_pars = encode(dict_par)
encoded_pars = encoded_pars.astype(float)
decoded_pars = decode(encoded_pars, dict_par)

In [20]:
# Encoded and decoded dictionaries

encoded_dict = encode(dict_var)

decoded_dict = decode(encoded_dict, dict_var)

In [21]:
# Define min and max values allowed

minVals = {'genActPower': np.array(np.zeros((1, genActPower.ravel().shape[0]))),
           'genExcActPower': np.array(np.zeros((1, genExcActPower.ravel().shape[0]))),
           'pImp': np.array(np.zeros((1, pImp.ravel().shape[0]))),
           'pExp': np.array(np.zeros((1, pExp.ravel().shape[0]))),
           'loadRedActPower': np.array(np.zeros((1, loadRedActPower.ravel().shape[0]))),
           'loadCutActPower':  np.array(np.zeros((1, loadCutActPower.ravel().shape[0]))),
           'loadENS': np.array(np.zeros((1, loadENS.ravel().shape[0]))),
           'storDchActPower': np.array(np.zeros((1, storDchActPower.ravel().shape[0]))),
           'storChActPower': np.array(np.zeros((1, storChActPower.ravel().shape[0]))),
           'EminRelaxStor': np.array(np.zeros((1, storEminRelaxStor.ravel().shape[0]))),
           'storEnerState': np.array(np.zeros((1, storEnerState.ravel().shape[0]))),
           'v2gDchActPower': np.array(np.zeros((1, v2gDchActPower.ravel().shape[0]))),
           'v2gChActPower': np.array(np.zeros((1, v2gChActPower.ravel().shape[0]))),
           'EminRelaxEV': np.array(np.zeros((1, v2gEminRelaxEv.ravel().shape[0]))),
           'v2gEnerState': np.array(np.zeros((1, v2gEnerState.ravel().shape[0]))),
           'csActPower': np.array(np.zeros((1, csActPower.ravel().shape[0]))),
           'csActPowerNet': np.array(np.zeros((1, csActPowerNet.ravel().shape[0]))),
           'genXo': np.array(np.zeros((1, genXo.ravel().shape[0]))),
           'loadXo': np.array(np.zeros((1, loadXo.ravel().shape[0]))),
           'storDchXo': np.array(np.zeros((1, storDchXo.ravel().shape[0]))),
           'storChXo': np.array(np.zeros((1, storChXo.ravel().shape[0]))),
           'v2gDchXo': np.array(np.zeros((1, v2gDchXo.ravel().shape[0]))),
           'v2gChXo': np.array(np.zeros((1, v2gChXo.ravel().shape[0])))}#,
           #'storSegSoC': np.array(np.zeros((1, storSegSoC.ravel().shape[0]))),
           #'v2gSegSoC': np.array(np.zeros((1, v2gSegSoC.ravel().shape[0]))),
           #'storSegXo': np.array(np.zeros((1, storSegXo.ravel().shape[0]))),
           #'v2gSegXo': np.array(np.zeros((1, v2gSegXo.ravel().shape[0])))}
encoded_minvals = encode(minVals)
encoded_minvals = encoded_minvals.astype(float)

maxVals = {'genActPower': genLimit[:, :, 0],
           'genExcActPower': genLimit[:, :, 0],
           'pImp': pMaxImp.ravel(),
           'pExp': pMaxExp.ravel(),
           'loadRedActPower': loadLimit[:, :, 2].ravel(),
           'loadCutActPower': loadLimit[:, :, 3].ravel(),
           'loadENS': decoded_pars['loadActPower'].ravel(),
           #'loadENS': 100*np.array(np.ones((1, loadENS.ravel().shape[0]))),
           'storDchActPower': storLimit[:, :, 1].ravel(),
           'storChActPower': storLimit[:, :, 0].ravel(),
           'EminRelaxStor': 20*np.array(np.ones((1, storEminRelaxStor.ravel().shape[0]))),
           'storEnerState': 100*np.array(np.ones((1, storEnerState.ravel().shape[0]))),
           'v2gDchActPower': 100*np.array(np.ones((1, v2gDchActPower.ravel().shape[0]))),
           'v2gChActPower': 100*np.array(np.ones((1, v2gChActPower.ravel().shape[0]))),
           'EminRelaxEV': 20*np.array(np.ones((1, v2gEminRelaxEv.ravel().shape[0]))),
           'v2gEnerState': 100*np.array(np.ones((1, v2gEnerState.ravel().shape[0]))),
           'csActPower': 20*np.array(np.ones((1, csActPower.ravel().shape[0]))),
           'csActPowerNet': 20*np.array(np.ones((1, csActPowerNet.ravel().shape[0]))),
           'genXo': np.array(np.ones((1, genXo.ravel().shape[0]))),
           'loadXo': np.array(np.ones((1, loadXo.ravel().shape[0]))),
           'storDchXo': np.array(np.ones((1, storDchXo.ravel().shape[0]))),
           'storChXo': np.array(np.ones((1, storChXo.ravel().shape[0]))),
           'v2gDchXo': np.array(np.ones((1, v2gDchXo.ravel().shape[0]))),
           'v2gChXo': np.array(np.ones((1, v2gChXo.ravel().shape[0])))}#,
           #'storSegSoC': 100*np.array(np.ones((1, storSegSoC.ravel().shape[0]))),
           #'v2gSegSoC': 100*np.array(np.ones((1, v2gSegSoC.ravel().shape[0]))),
           #'storSegXo': np.array(np.ones((1, storSegXo.ravel().shape[0]))),
           #'v2gSegXo': np.array(np.ones((1, v2gSegXo.ravel().shape[0])))}
encoded_maxvals = encode(maxVals)
encoded_maxvals = encoded_maxvals.astype(float)

In [22]:
# Define initial solution

sol = {'genActPower': genLimit[:, :, 0].ravel(),
       'genExcActPower': np.array(np.zeros((1, genExcActPower.ravel().shape[0]))),
       'pImp': pMaxImp.ravel(),
       'pExp': np.array(np.zeros((1, pExp.ravel().shape[0]))),
       'loadRedActPower': np.array(np.zeros((1, loadRedActPower.ravel().shape[0]))),
       'loadCutActPower':  np.array(np.zeros((1, loadCutActPower.ravel().shape[0]))),
       'loadENS': np.array(np.zeros((1, loadENS.ravel().shape[0]))),
       'storDchActPower': np.array(np.zeros((1, storDchActPower.ravel().shape[0]))),
       'storChActPower': np.array(np.zeros((1, storChActPower.ravel().shape[0]))),
       'EminRelaxStor': np.array(np.zeros((1, storEminRelaxStor.ravel().shape[0]))),
       'storEnerState': np.array(np.zeros((1, storEnerState.ravel().shape[0]))),
       'v2gDchActPower': np.array(np.zeros((1, v2gDchActPower.ravel().shape[0]))),
       'v2gChActPower': np.array(np.zeros((1, v2gChActPower.ravel().shape[0]))),
       'EminRelaxEV': np.array(np.zeros((1, v2gEminRelaxEv.ravel().shape[0]))),
       'v2gEnerState': np.array(np.zeros((1, v2gEnerState.ravel().shape[0]))),
       'csActPower': np.array(np.zeros((1, csActPower.ravel().shape[0]))),
       'csActPowerNet': np.array(np.zeros((1, csActPowerNet.ravel().shape[0]))),
       'genXo': np.array(np.zeros((1, genXo.ravel().shape[0]))),
       'loadXo': np.array(np.zeros((1, loadXo.ravel().shape[0]))),
       'storDchXo': np.array(np.ones((1, storDchXo.ravel().shape[0]))),
       'storChXo': np.array(np.zeros((1, storChXo.ravel().shape[0]))),
       'v2gDchXo': np.array(np.zeros((1, v2gDchXo.ravel().shape[0]))),
       'v2gChXo': np.array(np.zeros((1, v2gChXo.ravel().shape[0])))}#,
       #'storSegSoC': np.array(np.zeros((1, storSegSoC.ravel().shape[0]))),
       #'v2gSegSoC': np.array(np.zeros((1, v2gSegSoC.ravel().shape[0]))),
       #'storSegXo': np.array(np.zeros((1, storSegXo.ravel().shape[0]))),
       #'v2gSegXo': np.array(np.zeros((1, v2gSegXo.ravel().shape[0])))}

encoded_sol = encode(sol)
encoded_sol = encoded_sol.astype(float)

In [23]:
# Properly shaped initial Solution

sol_shaped = {'genActPower': genLimit[:, :, 0],
              'genExcActPower': np.zeros(genExcActPower.shape),
              'pImp': pMaxImp,
              'pExp': np.zeros(pExp.shape),
              'loadRedActPower': np.zeros(loadRedActPower.shape),
              'loadCutActPower': np.zeros(loadCutActPower.shape),
              'loadENS': np.zeros(loadENS.shape),
              'storDchActPower': np.zeros(storDchActPower.shape),
              'storChActPower': np.zeros(storChActPower.shape),
              'EminRelaxStor': np.zeros(storEminRelaxStor.shape),
              'storEnerState': np.zeros((storEnerState.shape)),
              'v2gDchActPower': np.zeros(v2gDchActPower.shape),
              'v2gChActPower': np.zeros(v2gChActPower.shape),
              'EminRelaxEV': np.zeros(v2gEminRelaxEv.shape),
              'v2gEnerState': np.zeros(v2gEnerState.shape),
              'csActPower': np.zeros(csActPower.shape),
              'csActPowerNet': np.zeros(csActPowerNet.shape),
              'genXo': np.zeros(genXo.shape),
              'loadXo': np.zeros(loadXo.shape),
              'storDchXo': np.ones(storDchXo.shape),
              'storChXo': np.zeros(storChXo.shape),
              'v2gDchXo': np.zeros(v2gDchXo.shape),
              'v2gChXo': np.zeros(v2gChXo.shape)}

encoded_shaped = encode(sol_shaped)
decoded_shaped = decode(encoded_shaped, dict_var)

In [24]:
# Define problem and obj function

class CustomProblem(Problem):
    
    def __init__(self, parameters, variables, xl, xu, n_var):
        super().__init__(n_var=n_var, n_obj=1, n_ieq_constr=0, xl=xl, xu=xu, vtype=float)
        
        # Set initial values
        self.__initial_variables__ = variables
        self.__var_idx__ = [variables[v].ravel().shape[0] for v in variables.keys()]
        self.__var_names__ = list(variables.keys())
        
        self.encoded = None
        self.decoded = None
        
        self.parameterAssign(parameters)
        
    
    def parameterAssign(self, parameters):
        self.genLimit = parameters['genLimit']
        self.genInfo = parameters['genInfo']
        self.pImpMax = parameters['pImpMax']
        self.pExpMax = parameters['pExpMax']
        self.loadLimit = parameters['loadLimit']
        self.loadActPower = parameters['loadActPower']
        self.storLimit = parameters['storLimit']
        self.storInfo = parameters['storInfo']
        self.v2gLimit = parameters['v2gLimit']
        self.v2gInfo = parameters['v2gInfo']
        self.csLimit = parameters['csLimit']
        self.csInfo = parameters['csInfo']
        self.EV_CS_Info = parameters['EV_CS_Info']
        self.buyPrice = parameters['buyPrice']
        self.sellPrice = parameters['sellPrice']
        self.t = parameters['t']
        self.gen = parameters['gen']
        self.load = parameters['load']
        self.stor = parameters['stor']
        self.v2g = parameters['v2g']
        self.cs = parameters['cs']
        #self.storPieceSegments = parameters['storPieceSegments']
        self.storCapCost = parameters['storCapCost']
        #self.storCyclesMult = parameters['storCyclesMult']
        self.v2gCapCost = parameters['v2gCapCost']
        #self.v2gCyclesMult = parameters['v2gCyclesMult']
        
        return
    
        
    def objectiveFunction(self, decoded):
        i: cython.int
        t: cython.int
        g: cython.int
        l: cython.int
        s: cython.int
        v: cython.int
            
        t_range: cython.int[self.t.shape[0]] = range(self.t.shape[0])
        gen_range: cython.int[self.gen.shape[0]] = range(self.gen.shape[0])
        load_range: cython.int[self.load.shape[0]] = range(self.load.shape[0])
        stor_range: cython.int[self.stor.shape[0]] = range(self.stor.shape[0])
        v2g_range: cython.int[self.v2g.shape[0]] = range(self.v2g.shape[0])
            
        balance_gens: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_loads: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_stor: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_v2g: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
            
        #for t in t_range:
        #    balance_gens[t] = np.sum([decoded['genActPower'][g, t] - decoded['genExcActPower'][g, t]
        #                              for g in gen_range])
        #    
        #    balance_loads[t] = np.sum([self.loadActPower[l, t] - decoded['loadRedActPower'][l, t] - decoded['loadCutActPower'][l, t] - decoded['loadENS'][l, t] 
        #                               for l in load_range])
        #    
        #    balance_stor[t] = np.sum([decoded['storChActPower'][s, t] - decoded['storDchActPower'][s, t]
        #                              for s in stor_range])
        #    
        #    balance_v2g[t] = np.sum([decoded['v2gChActPower'][v, t] - decoded['v2gDchActPower'][v, t]
        #                             for v in v2g_range])
        #    
        #balance_rest: cython.float[self.t.shape[0]] = balance_gens - balance_loads - balance_stor - balance_v2g
        #    
        ## Balance and import/export penalties
        #mask = balance_rest > 0
        #decoded['pImp'][mask] *= 0
        #decoded['pExp'][mask] = balance_rest[mask]
        #
        #mask = balance_rest < 0
        #decoded['pExp'][mask] *= 0
        #decoded['pImp'][mask] = abs(balance_rest)[mask]
        
        
        # Attribute penalties for import/export
        balance_penalty: cython.float = 0
        for t in t_range:
            if (decoded['pImp'][t] > self.pImpMax[t]):
                balance_penalty += 100000
                
            if (decoded['pExp'][t] > self.pExpMax[t]):
                balance_penalty += 100000
                
            
        temp_gens: cython.float = sum([decoded['genActPower'][g, t] * self.genLimit[g, t, 2] + decoded['genExcActPower'][g, t] * self.genLimit[g, t, 4]
                                       for t in t_range for g in gen_range])
            
        temp_loads: cython.float = sum([decoded['loadRedActPower'][l, t] * self.loadLimit[l, t, 6] + decoded['loadCutActPower'][l, t] * self.loadLimit[l, t, 7] + decoded['loadENS'][l, t] * self.loadLimit[l, t, 9] 
                                        for t in t_range for l in load_range])
            
        #temp_stor: cython.float = sum([self.storDchActPower[s, t] * self.storLimit[s, t, 3] - self.storChActPower[s, t] * self.storLimit[s, t, 2] + self.EminRelaxStor[s, t] * 200 
        #                               for t in t_range for s in stor_range])
            
        #temp_v2g: cython.float = sum([self.v2gDchActPower[v, t] * self.v2gLimit[v, t, 6] - self.v2gChActPower[v, t] * self.v2gLimit[v, t, 5] + self.EminRelaxEv[v, t] * 200
        #                              for t in t_range for v in v2g_range])
            
        temp_rest: cython.float = sum([decoded['pImp'][t] * self.buyPrice[t] - decoded['pExp'][t] * self.sellPrice[t] 
                                       for t in t_range])   
        
        temp_storDeg = sum([self.storCapCost[s] * (decoded['storEnerState'][s, t]/self.storInfo[s, 5] - 0.63)**2 + decoded['storDchActPower'][s, t] * self.storLimit[s, t, 3] + decoded['storChActPower'][s, t] * self.storLimit[s, t, 2] + (6.5e-3)/self.storInfo[s, 5] * decoded['storChActPower'][s, t]**2
                            for t in t_range for s in stor_range])
        
        temp_v2gDeg = sum([self.v2gCapCost[v] * (decoded['v2gEnerState'][v, t]/self.v2gInfo[v, 4] - 0.63)**2 + decoded['v2gDchActPower'][v, t] * self.v2gLimit[v, t, 6] + decoded['v2gChActPower'][v, t] * self.v2gLimit[v, t, 5] + (6.5e-3)/self.v2gInfo[v, 4] * decoded['v2gChActPower'][v, t]**2
                           for t in t_range for v in v2g_range])
            
        #temp_storDeg: cython.float = sum([self._auxStorDegCost(s, t) for t in t_range for s in stor_range])
            
        #temp_v2gDeg: cython.float = sum([self._auxV2GDegCost(v, t) for t in t_range for v in v2g_range])
        
        #self.objFn = (temp_gens + temp_loads + temp_stor + temp_v2g + temp_rest)*1.0 + balance_penalty + temp_storDeg# + temp_v2gDeg
        self.objFn = temp_gens + temp_loads + temp_rest + balance_penalty + temp_storDeg + temp_v2gDeg

        return
    
    
    def decode(self, new_variables):
        #print(new_variables.shape)
        result_decoded = {}
        current_index = 0
        
        for idx in range(len(self.__var_names__)):
            result_index = current_index + self.__var_idx__[idx]
            result_decoded[self.__var_names__[idx]] = np.reshape(new_variables[current_index:result_index],
                                                                 self.__initial_variables__[self.__var_names__[idx]].shape)
            
            current_index = result_index
        
        
        return result_decoded
    
    
    def _evaluate(self, x, out, *args, **kwargs):
        objFn = []
        for temp in range(x.shape[0]):
            temp_solution = self.decode(x[temp])
            temp_obj = self.objectiveFunction(temp_solution)
            #self.objectiveFuction
            objFn.append(self.objFn)
            
        out['F'] = objFn

In [25]:
# Define custom repair class

class CustomRepair(Repair):
    
    def __init__(self, parameters, variables):
        super().__init__()
        
        # Set initial values
        self.__initial_variables__ = variables
        self.__var_idx__ = [variables[v].ravel().shape[0] for v in variables.keys()]
        self.__var_names__ = list(variables.keys())
        
        # Value assignments
        self.parameterAssign(parameters)
        #self.variableAssign(variables)
        
        return
        
        
    def parameterAssign(self, parameters):
        self.genLimit = parameters['genLimit']
        self.genInfo = parameters['genInfo']
        self.pImpMax = parameters['pImpMax']
        self.pExpMax = parameters['pExpMax']
        self.loadLimit = parameters['loadLimit']
        self.loadActPower = parameters['loadActPower']
        self.storLimit = parameters['storLimit']
        self.storInfo = parameters['storInfo']
        self.v2gLimit = parameters['v2gLimit']
        self.v2gInfo = parameters['v2gInfo']
        self.csLimit = parameters['csLimit']
        self.csInfo = parameters['csInfo']
        self.EV_CS_Info = parameters['EV_CS_Info']
        self.buyPrice = parameters['buyPrice']
        self.sellPrice = parameters['sellPrice']
        self.t = parameters['t']
        self.gen = parameters['gen']
        self.load = parameters['load']
        self.stor = parameters['stor']
        self.v2g = parameters['v2g']
        self.cs = parameters['cs']
        #self.storPieceSegments = parameters['storPieceSegments']
        self.storCapCost = parameters['storCapCost']
        #self.storCyclesMult = parameters['storCyclesMult']
        self.v2gCapCost = parameters['v2gCapCost']
        #self.v2gCyclesMult = parameters['v2gCyclesMult']
        
        return
    
    
    def checkImpExp(self, decoded):
        # Imports
        mask = decoded['pImp'] >= self.pImpMax
        decoded['pImp'][mask] = self.pImpMax[mask]
        
        mask = decoded['pImp'] < 0
        decoded['pImp'][mask] *= 0
        
        # Exports
        mask = decoded['pExp'] > self.pExpMax
        decoded['pExp'][mask] = self.pExpMax[mask]
        
        mask = decoded['pExp'] < 0
        decoded['pExp'][mask] *= 0
        
        return
    
    
    def checkGens(self, decoded):
        decoded['genXo'] = (decoded['genXo'] > 0.5).astype(int)
        
        # Maximum generation
        mask = decoded['genActPower'] > self.genLimit[:, :, 0]
        decoded['genActPower'][mask] = self.genLimit[:, :, 0][mask]
        
        mask = decoded['genExcActPower'] > self.genLimit[:, :, 0]
        decoded['genExcActPower'][mask] = self.genLimit[:, :, 0][mask]
        
        mask = decoded['genActPower'] < 0
        decoded['genActPower'][mask] *= 0
        
        decoded['genExcActPower'] = np.zeros(decoded['genExcActPower'].shape)
    
        
        # Generator types - type 1
        mask = self.genInfo[:, 1] == np.ones((self.genInfo[:, 4].shape))
        decoded['genActPower'][mask] = (self.genLimit[:, :, 0] * decoded['genXo'])[mask]
        #decoded['genExcActPower'][mask] *= 0
        
        # Generator types - type 2
        mask = self.genInfo[:, 1] == 2*np.ones((self.genInfo[:, 4].shape))
        decoded['genExcActPower'][mask] = self.genLimit[:, :, 0][mask] - decoded['genActPower'][mask]
        
        #mask = decoded['genExcActPower'] < 0
        #decoded['genExcActPower'][mask] *= 0
    
        return
    
    
    def checkLoads(self, decoded):
        
        # Bound the values to either 0 or 1
        decoded['loadXo'] = (decoded['loadXo'] > 0.5).astype(int)
        
        # Build a matrix of ones
        temp_vals = np.ones((decoded['loadRedActPower'].shape[0],
                             decoded['loadRedActPower'].shape[1]))
        
        # RedAct value checks
        mask = decoded['loadRedActPower'] > temp_vals * self.loadLimit[:, :, 2]
        decoded['loadRedActPower'][mask] = (temp_vals * self.loadLimit[:, :, 2])[mask]
        
        mask = decoded['loadRedActPower'] < 0
        decoded['loadRedActPower'][mask] *= 0
        
        # loadCut value checks
        decoded['loadCutActPower'] = self.loadLimit[:, :, 3] * decoded['loadXo']
        #self.loadCutActPower *= self.loadXo
        
        # loadENS
        temp_vals = self.loadActPower - decoded['loadRedActPower'] - decoded['loadCutActPower']
        mask = decoded['loadENS'] > temp_vals
        decoded['loadENS'][mask] = temp_vals[mask]
        #decoded['loadENS'] = temp_vals
        return
    
    
    def checkStor(self, decoded):
        
        # Binary variables bound
        decoded['storChXo'] = (decoded['storChXo'] > 0.5).astype(int)
        decoded['storDchXo'] = (decoded['storDchXo'] > 0.5).astype(int)
        
        mask = None
        # Discharge inequality
        mask = decoded['storDchActPower'] > self.storLimit[:, :, 1]
        decoded['storDchActPower'][mask] = self.storLimit[:, :, 1][mask]
        
        mask = decoded['storDchActPower'] <= 0
        decoded['storDchActPower'][mask] *= 0
        
        # Charge inequality
        mask = decoded['storChActPower'] > self.storLimit[:, :, 0]
        decoded['storChActPower'][mask] = self.storLimit[:, :, 0][mask]
        
        mask = decoded['storChActPower'] <= 0
        decoded['storChActPower'][mask] *= 0
        
        # Initial stor SoC
        decoded['storEnerState'][:, 0] = self.storInfo[:, 5] * (self.storInfo[:, 9]/100) + decoded['storChActPower'][:, 0] * (self.storInfo[:, 7]/100) - decoded['storDchActPower'][:, 0]/(self.storInfo[:, 8]/100)
        
        t: cython.int
        t_range: cython.int[len(self.t) - 1] = range(1, len(self.t))
        # Fix the timestep dependencies
        for t in t_range:
            # Check if charging
            mask = decoded['storChXo'][:, t] > np.zeros(decoded['storChXo'][:, t].shape)
            charged = decoded['storChActPower'][:, t] * (self.storInfo[:, 7]/100)
            # Prevent charging beyong limit
            secondary_mask = (decoded['storEnerState'][:, t-1] + charged) > self.storInfo[:, 5]
            decoded['storChActPower'][:, t][secondary_mask] = ((self.storInfo[:, 5] - decoded['storEnerState'][:, t-1]) / (self.storInfo[:, 7]/100))[secondary_mask]
            
            # Check if discharging
            mask = decoded['storDchXo'][:, t] > np.zeros(decoded['storDchXo'][:, t].shape)
            discharged = decoded['storDchActPower'][:, t] / (self.storInfo[:, 8]/100)
            secondary_mask = (decoded['storEnerState'][:, t-1] - discharged) < 0
            decoded['storDchActPower'][:, t][secondary_mask] = (decoded['storEnerState'][:, t-1] * (self.storInfo[:, 8]/100))[secondary_mask]
            
            decoded['storChActPower'][:, t] *= decoded['storChXo'][:, t]
            decoded['storDchActPower'][:, t] *= decoded['storDchXo'][:, t]
            decoded['storEnerState'][:, t] = decoded['storEnerState'][:, t-1] + (decoded['storChActPower'][:, t] * (self.storInfo[:, 7]/100)) - (decoded['storDchActPower'][:, t]/(self.storInfo[:, 8]/100))
            
            mask = decoded['storEnerState'][:, t] < self.storInfo[:, 6] * (self.storInfo[:, 5] / 100) - decoded['EminRelaxStor'][:, t]
            decoded['storEnerState'][:, t][mask] = (self.storInfo[:, 6] * (self.storInfo[:, 5] / 100) - decoded['EminRelaxStor'][:, t])[mask]
        return
    
    
    def checkV2G(self, decoded):
        # Binary variables bounding
        #self.v2gChXo = (self.v2gDchXo < 0.33).astype(int)
        #self.v2gDchXo = (self.v2gDchXo > 0.66).astype(int)
        
        decoded['v2gDchActPower'] = np.zeros((len(self.v2g),
                                              len(self.t)))
        
        decoded['v2gChActPower'] = np.zeros((len(self.v2g),
                                             len(self.t)))
        
        decoded['v2gEnerState'] = np.zeros((len(self.v2g),
                                            len(self.t)))
        
        decoded['EminRelaxEV'] = np.zeros((len(self.v2g),
                                           len(self.t)))
        
        # Bound binary variables
        decoded['v2gChXo'] = (decoded['v2gChXo'] > 0.5).astype(int)
        decoded['v2gDchXo'] = (decoded['v2gDchXo'] > 0.5).astype(int)
        #self.v2gChXo = 1 - self.v2gDchXo

        mask = None

        v: cython.int
        c: cython.int

        # Discharge rate
        #temp_val = par['v2gLimit'][:, :, 4] * par['v2gLimit'][:, :, 0]
        #mask = pop['v2gDchActPower'] > temp_val
        #pop['v2gDchActPower'][mask] = temp_val[mask]


        # Charge rate
        #temp_val = par['v2gLimit'][:, :, 3] * par['v2gLimit'][:, :, 0]
        #mask = pop['v2gChActPower'] > temp_val
        #pop['v2gChActPower'][mask] = temp_val[mask]

        decoded['v2gDchActPower'] = self.v2gLimit[:, :, 4] * self.v2gLimit[:, :, 0]
        decoded['v2gChActPower'] = self.v2gLimit[:, :, 3] * self.v2gLimit[:, :, 0]

        # V2G constraints
        v_range: cython.int[len(self.v2g)] = range(len(self.v2g))
        c_range: cython.int[len(self.cs)] = range(len(self.cs))
        for v in v_range:
            # Check connection to charging stations
            isConnected = False
            connectedTo = 0

            # Check the charging stations
            for c in c_range:
                isConnected = True
                if self.EV_CS_Info[v, c, 0] > 0:
                    decoded['v2gChActPower'][v, 0] = min(decoded['v2gChActPower'][v, 0], self.csInfo[c, 4])
                    decoded['v2gDchActPower'][v, 0] = min(decoded['v2gDchActPower'][v, 0], self.csInfo[c, 5])

                    connectedTo = c
                else:
                    decoded['v2gDchXo'][v, 0] = 0
                    decoded['v2gChXo'][v, 0] = 0

            if decoded['v2gChXo'][v, 0] + decoded['v2gDchXo'][v, 0] > 1:
                decoded['v2gDchXo'][v, 0] = 1 - decoded['v2gChXo'][v, 0]

            mask = self.v2gLimit[v, :, 2] > 0
            temp_val = self.v2gLimit[v, :, 2][mask]

            if isConnected & (len(temp_val) > 0):
                if decoded['v2gEnerState'][v, 0] < temp_val[0]:
                    next_index = next((idx for idx, val in np.ndenumerate(self.v2gLimit[v, :, 2]) 
                                       if val==temp_val[0]))[0]
                    min_tsteps = np.ceil((temp_val[0] - decoded['v2gEnerState'][v, 0]) / self.csInfo[connectedTo, 4]) - 1

                    if min_tsteps >= next_index:
                        decoded['v2gChXo'][v, 0] = 1
                        decoded['v2gDchXo'][v, 0] = 0

            decoded['v2gChActPower'][v, 0] *= decoded['v2gChXo'][v,  0]
            decoded['v2gDchActPower'][v, 0] *= decoded['v2gDchXo'][v, 0]

            if self.v2gLimit[v, 0, 0] == 0:
                decoded['v2gEnerState'][v, 0] = 0
            elif self.v2gLimit[v, 0, 0] == 1:
                decoded['v2gEnerState'][v, 0] = self.v2gLimit[v, 0, 1] + decoded['v2gChActPower'][v, 0] * self.v2gInfo[v, 7] - decoded['v2gDchActPower'][v, 0] / self.v2gInfo[v, 8]

            #print('v2gEnerState: {}'.format(pop['v2gEnerState'][v, 0]))

            # Timestep
            for t in range(1, len(self.t)):

                isConnected = False
                connectedTo = 0

                # Check the charging stations
                for c in c_range:
                    if self.EV_CS_Info[v, c, t] > 0:
                        isConnected = True
                        decoded['v2gChActPower'][v, t] = min(decoded['v2gChActPower'][v, t], self.csInfo[c, 4])
                        decoded['v2gDchActPower'][v, t] = min(decoded['v2gDchActPower'][v, t], self.csInfo[c, 5])

                        connectedTo = c
                    else:
                        decoded['v2gDchXo'][v, t] = 0
                        decoded['v2gChXo'][v, t] = 0


                # Disable charge and discharge in the same period
                if decoded['v2gChXo'][v, t] + decoded['v2gDchXo'][v, t] > 1:
                    decoded['v2gDchXo'][v, t] = 1 - decoded['v2gChXo'][v, t]

                # Incentivise charge to meet minimum limits
                mask = self.v2gLimit[v, t:, 2] > 0
                temp_val = self.v2gLimit[v, t:, 2][mask]

                # Check if there are any requirements for EVs
                if isConnected & (len(temp_val) > 0):
                    if decoded['v2gEnerState'][v, t-1] < temp_val[0]:
                        next_index = next((idx for idx, val in np.ndenumerate(self.v2gLimit[v, t:, 2]) 
                                           if val==temp_val[0]))[0]
                        min_tsteps = np.ceil((temp_val[0] - decoded['v2gEnerState'][v, t-1])  /  self.csInfo[connectedTo, 4]) - 1
                        if min_tsteps <= next_index:
                            decoded['v2gChXo'][v, t] = 1
                            decoded['v2gDchXo'][v, t] = 0



                            if (decoded['v2gEnerState'][v, t-1] + (decoded['v2gChActPower'][v, t] * float(self.v2gInfo[v, 7]))) >= self.v2gInfo[v, 4]:
                                decoded['v2gChActPower'][v, t] = (self.v2gInfo[v, 4] - decoded['v2gEnerState'][v, t-1]) / float(self.v2gInfo[v, 7])

                # Prevent charging when battery is full
                if decoded['v2gEnerState'][v, t-1] == self.v2gInfo[v, 4]:
                    #print('HERE')
                    decoded['v2gChXo'][v, t] = 0

                # Prevent discharge when battery is empty
                elif decoded['v2gEnerState'][v, t-1] == 0:
                    decoded['v2gDchXo'][v, t] = 0

                decoded['v2gChActPower'][v, t] *= decoded['v2gChXo'][v, t]
                decoded['v2gDchActPower'][v, t] *= decoded['v2gDchXo'][v, t]

                # Update battery capacity
                if (self.v2gLimit[v, t-1, 0] == 1) & (self.v2gLimit[v, t, 0] == 1):
                    decoded['v2gEnerState'][v, t] = decoded['v2gEnerState'][v, t-1] + self.v2gLimit[v, t, 1] + (decoded['v2gChActPower'][v, t] * float(self.v2gInfo[v, 7])) - (decoded['v2gDchActPower'][v, t] / float(self.v2gInfo[v, 8]))
                elif (self.v2gLimit[v, t-1, 0] == 0) & (self.v2gLimit[v, t, 0] == 1):
                    decoded['v2gEnerState'][v, t] = self.v2gLimit[v, t, 1] + (decoded['v2gChActPower'][v, t] * float(self.v2gInfo[v, 7])) + (decoded['v2gDchActPower'][v, t] / float(self.v2gInfo[v, 8]))

        return
    
    
    def checkCS(self, decoded):
        
        decoded['csActPower'] = np.zeros((len(self.cs),
                                          len(self.t)))
        
        decoded['csActPowerNet'] = np.zeros((len(self.cs),
                                             len(self.t)))
        
        #self.v2gChXo = (self.v2gDchXo < 0.33).astype(int)
        #self.v2gDchXo = (self.v2gDchXo > 0.66).astype(int)
        
        
        c: cython.int
        t: cython.int
        v: cython.int
            
        t_range: cython.int[len(self.t)] = range(len(self.t))
        c_range: cython.int[len(self.cs)] = range(len(self.cs))
        v_range: cython.int[len(self.v2g)] = range(len(self.v2g))
        
        # Timesteps
        for t in t_range:

            # Charging station constraints
            for c in c_range:

                temp_val = 0
                temp_val2 = 0
                for v in v_range:
                    if self.EV_CS_Info[v, c, t] > 0:
                        temp_val += (decoded['v2gChActPower'][v, t]  - decoded['v2gDchActPower'][v, t])
                        temp_val2 += (decoded['v2gChActPower'][v, t] / (self.csInfo[c, 6] / 100) - (decoded['v2gDchActPower'][v, t] * self.csInfo[c, 7] / 100))


                if temp_val > self.csInfo[c, 4]:
                    temp_val = self.csInfo[c, 4]
                if temp_val < -self.csInfo[c, 5]:
                    temp_val = -self.csInfo[c, 5]

                decoded['csActPower'][c, t] = temp_val
                decoded['csActPowerNet'][c, t] = temp_val2
        return
    
    
    def checkBalance(self, decoded):
        i: cython.int
        t: cython.int
        g: cython.int
        l: cython.int
        s: cython.int
        v: cython.int
            
        t_range: cython.int[self.t.shape[0]] = range(self.t.shape[0])
        gen_range: cython.int[self.gen.shape[0]] = range(self.gen.shape[0])
        load_range: cython.int[self.load.shape[0]] = range(self.load.shape[0])
        stor_range: cython.int[self.stor.shape[0]] = range(self.stor.shape[0])
        v2g_range: cython.int[self.v2g.shape[0]] = range(self.v2g.shape[0])
        cs_range: cython.int[self.cs.shape[0]] = range(self.cs.shape[0])
            
        balance_gens: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_loads: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_stor: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_v2g: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
        balance_cs: cython.float[self.t.shape[0]] = np.zeros(self.t.shape[0])
            
        for t in t_range:
            balance_gens[t] = np.sum([decoded['genActPower'][g, t] - decoded['genExcActPower'][g, t]
                                      for g in gen_range])
            
            balance_loads[t] = np.sum([self.loadActPower[l, t] - decoded['loadRedActPower'][l, t] - decoded['loadCutActPower'][l, t] - decoded['loadENS'][l, t] 
                                       for l in load_range])
            
            balance_stor[t] = np.sum([decoded['storChActPower'][s, t] - decoded['storDchActPower'][s, t]
                                      for s in stor_range])
            
            #balance_v2g[t] = np.sum([decoded['csActPower'][v, t] - decoded['v2gDchActPower'][v, t]
            #                         for v in v2g_range])
            balance_cs[t] = np.sum([decoded['csActPowerNet'][c, t]
                                    for c in cs_range])
            
        balance_rest: cython.float[self.t.shape[0]] = balance_gens - balance_loads - balance_stor - balance_cs
            
        # Balance and import/export penalties
        mask = balance_rest > 0
        decoded['pImp'][mask] *= 0
        decoded['pExp'][mask] = balance_rest[mask]
        
        mask = balance_rest < 0
        decoded['pExp'][mask] *= 0
        decoded['pImp'][mask] = abs(balance_rest)[mask]
        
        return
    
    
    def _do(self, problem, Z, **kwargs):
        for z in np.arange(len(Z)):
            temp_z = self.decode(Z[z])
            self.checkImpExp(temp_z)
            self.checkGens(temp_z)
            self.checkStor(temp_z)
            self.checkV2G(temp_z)
            self.checkCS(temp_z)
            self.checkLoads(temp_z)
            self.checkBalance(temp_z)
            
            #print(temp_z.keys())
            
            temp_z = self.encode(temp_z)
            
            Z[z] = temp_z
        return Z
    
    
    def encode(self, decoded):
        temp = np.concatenate([decoded[k].ravel() for k in decoded.keys()])
        return temp
    
    
    def decode(self, new_variables):
        #print(new_variables.shape)
        result_decoded = {}
        current_index = 0
        
        for idx in range(len(self.__var_names__)):
            result_index = current_index + self.__var_idx__[idx]
            result_decoded[self.__var_names__[idx]] = np.reshape(new_variables[current_index:result_index],
                                                                 self.__initial_variables__[self.__var_names__[idx]].shape)
            
            current_index = result_index
        
        
        return result_decoded

In [26]:
algorithm = GA(pop_size=100,
               repair=CustomRepair(parameters=decoded_pars, 
                                                  variables=decoded_shaped))#,


# Default termination:
termination = DefaultSingleObjectiveTermination(xtol=1e-8,
                                                cvtol=1e-6,
                                                ftol=1e-6,
                                                period=1000,
                                                n_max_gen=5e4,
                                                n_max_evals=5e9)

customProblem = CustomProblem(parameters=decoded_pars,
                              variables=decoded_shaped,
                              xl=encoded_minvals,
                              xu=encoded_maxvals,
                              n_var=2424)

temp_res = minimize(customProblem,
                    algorithm,
                    termination=termination,
                    save_history=False,
                    verbose=True)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      100 |  1.684721E+05 |  4.162114E+03


c:\users\feel\jupyter\ecgomes\upacs_study\venv\lib\site-packages\pymoo\operators\crossover\sbx.py:47: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (eta + 1.0)))[mask]


     2 |      200 |  5.857117E+04 |  4.162114E+03
     3 |      300 |  4.741392E+03 |  3.944846E+03
     4 |      400 |  4.518809E+03 |  3.867347E+03
     5 |      500 |  4.436899E+03 |  3.867347E+03
     6 |      600 |  4.306050E+03 |  3.797912E+03
     7 |      700 |  4.197948E+03 |  3.732014E+03
     8 |      800 |  4.092642E+03 |  3.645065E+03
     9 |      900 |  4.015031E+03 |  3.477230E+03
    10 |     1000 |  3.941834E+03 |  3.477230E+03
    11 |     1100 |  3.846247E+03 |  3.477230E+03
    12 |     1200 |  3.781013E+03 |  3.477230E+03
    13 |     1300 |  3.724144E+03 |  3.477230E+03
    14 |     1400 |  3.673218E+03 |  3.342673E+03
    15 |     1500 |  3.641518E+03 |  3.342673E+03
    16 |     1600 |  3.589610E+03 |  3.291153E+03
    17 |     1700 |  3.528723E+03 |  3.254546E+03
    18 |     1800 |  3.457628E+03 |  3.207440E+03
    19 |     1900 |  3.403344E+03 |  3.156465E+03
    20 |     2000 |  3.345773E+03 |  3.138419E+03
    21 |     2100 |  3.289500E+03 |  3.098118E+03


   166 |    16600 |  1.933481E+03 |  1.929357E+03
   167 |    16700 |  1.932294E+03 |  1.929207E+03
   168 |    16800 |  1.931196E+03 |  1.929207E+03
   169 |    16900 |  1.930448E+03 |  1.929043E+03
   170 |    17000 |  1.929839E+03 |  1.928630E+03
   171 |    17100 |  1.929296E+03 |  1.926843E+03
   172 |    17200 |  1.928780E+03 |  1.925920E+03
   173 |    17300 |  1.928039E+03 |  1.918842E+03
   174 |    17400 |  1.927059E+03 |  1.915387E+03
   175 |    17500 |  1.925245E+03 |  1.913180E+03
   176 |    17600 |  1.922428E+03 |  1.913068E+03
   177 |    17700 |  1.917903E+03 |  1.902280E+03
   178 |    17800 |  1.912959E+03 |  1.894949E+03
   179 |    17900 |  1.908298E+03 |  1.894942E+03
   180 |    18000 |  1.901914E+03 |  1.894070E+03
   181 |    18100 |  1.899054E+03 |  1.893269E+03
   182 |    18200 |  1.896499E+03 |  1.890068E+03
   183 |    18300 |  1.894244E+03 |  1.889557E+03
   184 |    18400 |  1.892748E+03 |  1.888363E+03
   185 |    18500 |  1.891163E+03 |  1.887488E+03


   330 |    33000 |  1.646430E+03 |  1.643856E+03
   331 |    33100 |  1.645001E+03 |  1.643056E+03
   332 |    33200 |  1.644199E+03 |  1.643056E+03
   333 |    33300 |  1.643516E+03 |  1.631120E+03
   334 |    33400 |  1.642671E+03 |  1.631049E+03
   335 |    33500 |  1.641255E+03 |  1.630240E+03
   336 |    33600 |  1.638491E+03 |  1.628547E+03
   337 |    33700 |  1.632678E+03 |  1.622456E+03
   338 |    33800 |  1.629021E+03 |  1.617316E+03
   339 |    33900 |  1.627124E+03 |  1.617055E+03
   340 |    34000 |  1.624066E+03 |  1.615010E+03
   341 |    34100 |  1.620801E+03 |  1.610026E+03
   342 |    34200 |  1.617622E+03 |  1.610026E+03
   343 |    34300 |  1.614959E+03 |  1.607159E+03
   344 |    34400 |  1.612773E+03 |  1.603641E+03
   345 |    34500 |  1.610009E+03 |  1.602790E+03
   346 |    34600 |  1.607322E+03 |  1.598005E+03
   347 |    34700 |  1.603947E+03 |  1.597423E+03
   348 |    34800 |  1.600903E+03 |  1.594394E+03
   349 |    34900 |  1.597679E+03 |  1.591564E+03


...

  8694 |   869400 |  6.079285E+02 |  6.079285E+02
  8695 |   869500 |  6.079285E+02 |  6.079285E+02
  8696 |   869600 |  6.079285E+02 |  6.079285E+02
  8697 |   869700 |  6.079285E+02 |  6.079285E+02
  8698 |   869800 |  6.079285E+02 |  6.079285E+02
  8699 |   869900 |  6.079285E+02 |  6.079285E+02
  8700 |   870000 |  6.079285E+02 |  6.079285E+02
  8701 |   870100 |  6.079285E+02 |  6.079285E+02
  8702 |   870200 |  6.079285E+02 |  6.079285E+02
  8703 |   870300 |  6.079285E+02 |  6.079285E+02
  8704 |   870400 |  6.079285E+02 |  6.079285E+02
  8705 |   870500 |  6.079285E+02 |  6.079285E+02
  8706 |   870600 |  6.079285E+02 |  6.079285E+02
  8707 |   870700 |  6.079285E+02 |  6.079285E+02
  8708 |   870800 |  6.079285E+02 |  6.079285E+02
  8709 |   870900 |  6.079285E+02 |  6.079285E+02
  8710 |   871000 |  6.079285E+02 |  6.079285E+02
  8711 |   871100 |  6.079285E+02 |  6.079285E+02
  8712 |   871200 |  6.079285E+02 |  6.079285E+02
  8713 |   871300 |  6.079285E+02 |  6.079285E+02


  8858 |   885800 |  6.079285E+02 |  6.079285E+02
  8859 |   885900 |  6.079285E+02 |  6.079285E+02
  8860 |   886000 |  6.079285E+02 |  6.079285E+02
  8861 |   886100 |  6.079285E+02 |  6.079285E+02
  8862 |   886200 |  6.079285E+02 |  6.079285E+02
  8863 |   886300 |  6.079285E+02 |  6.079285E+02
  8864 |   886400 |  6.079285E+02 |  6.079285E+02
  8865 |   886500 |  6.079285E+02 |  6.079285E+02
  8866 |   886600 |  6.079285E+02 |  6.079285E+02
  8867 |   886700 |  6.079285E+02 |  6.079285E+02
  8868 |   886800 |  6.079285E+02 |  6.079285E+02
  8869 |   886900 |  6.079285E+02 |  6.079285E+02
  8870 |   887000 |  6.079285E+02 |  6.079285E+02
  8871 |   887100 |  6.079285E+02 |  6.079285E+02
  8872 |   887200 |  6.079285E+02 |  6.079285E+02
  8873 |   887300 |  6.079285E+02 |  6.079285E+02
  8874 |   887400 |  6.079285E+02 |  6.079285E+02
  8875 |   887500 |  6.079285E+02 |  6.079285E+02
  8876 |   887600 |  6.079285E+02 |  6.079285E+02
  8877 |   887700 |  6.079285E+02 |  6.079285E+02


  9022 |   902200 |  6.079285E+02 |  6.079285E+02
  9023 |   902300 |  6.079285E+02 |  6.079285E+02
  9024 |   902400 |  6.079285E+02 |  6.079285E+02
  9025 |   902500 |  6.079285E+02 |  6.079285E+02
  9026 |   902600 |  6.079285E+02 |  6.079285E+02
  9027 |   902700 |  6.079285E+02 |  6.079285E+02
  9028 |   902800 |  6.079285E+02 |  6.079285E+02
  9029 |   902900 |  6.079285E+02 |  6.079285E+02
  9030 |   903000 |  6.079285E+02 |  6.079285E+02
  9031 |   903100 |  6.079285E+02 |  6.079285E+02
  9032 |   903200 |  6.079285E+02 |  6.079285E+02
  9033 |   903300 |  6.079285E+02 |  6.079285E+02
  9034 |   903400 |  6.079285E+02 |  6.079285E+02
  9035 |   903500 |  6.079285E+02 |  6.079285E+02
  9036 |   903600 |  6.079285E+02 |  6.079285E+02
  9037 |   903700 |  6.079285E+02 |  6.079285E+02
  9038 |   903800 |  6.079285E+02 |  6.079285E+02
  9039 |   903900 |  6.079285E+02 |  6.079285E+02
  9040 |   904000 |  6.079285E+02 |  6.079285E+02
  9041 |   904100 |  6.079285E+02 |  6.079285E+02


  9186 |   918600 |  6.079285E+02 |  6.079285E+02
  9187 |   918700 |  6.079285E+02 |  6.079285E+02
  9188 |   918800 |  6.079285E+02 |  6.079285E+02
  9189 |   918900 |  6.079285E+02 |  6.079285E+02
  9190 |   919000 |  6.079285E+02 |  6.079285E+02
  9191 |   919100 |  6.079285E+02 |  6.079285E+02
  9192 |   919200 |  6.079285E+02 |  6.079285E+02
  9193 |   919300 |  6.079285E+02 |  6.079285E+02
  9194 |   919400 |  6.079285E+02 |  6.079285E+02
  9195 |   919500 |  6.079285E+02 |  6.079285E+02
  9196 |   919600 |  6.079285E+02 |  6.079285E+02
  9197 |   919700 |  6.079285E+02 |  6.079285E+02
  9198 |   919800 |  6.079285E+02 |  6.079285E+02
  9199 |   919900 |  6.079285E+02 |  6.079285E+02
  9200 |   920000 |  6.079285E+02 |  6.079285E+02
  9201 |   920100 |  6.079285E+02 |  6.079285E+02
  9202 |   920200 |  6.079285E+02 |  6.079285E+02
  9203 |   920300 |  6.079285E+02 |  6.079285E+02
  9204 |   920400 |  6.079285E+02 |  6.079285E+02
  9205 |   920500 |  6.079285E+02 |  6.079285E+02


  9350 |   935000 |  6.079285E+02 |  6.079285E+02
  9351 |   935100 |  6.079285E+02 |  6.079285E+02
  9352 |   935200 |  6.079285E+02 |  6.079285E+02
  9353 |   935300 |  6.079285E+02 |  6.079285E+02
  9354 |   935400 |  6.079285E+02 |  6.079285E+02
  9355 |   935500 |  6.079285E+02 |  6.079285E+02
  9356 |   935600 |  6.079285E+02 |  6.079285E+02
  9357 |   935700 |  6.079285E+02 |  6.079285E+02
  9358 |   935800 |  6.079285E+02 |  6.079285E+02
  9359 |   935900 |  6.079285E+02 |  6.079285E+02
  9360 |   936000 |  6.079285E+02 |  6.079285E+02
  9361 |   936100 |  6.079285E+02 |  6.079285E+02
  9362 |   936200 |  6.079285E+02 |  6.079285E+02


In [27]:
# Save the results

name = 'run10'

minimized_solution = pd.DataFrame(temp_res.X)
minimized_solution.to_csv('results/paper_ga/'+name+'/solution.csv')

minimized_stats = pd.DataFrame({'objFun': temp_res.F,
                                'runtime': temp_res.exec_time,
                                'gen': temp_res.algorithm.n_gen - 1})
minimized_stats.to_csv('results/paper_ga/'+name+'/stats.csv')